## Исследование данных сервиса "Яндекс.Музыка"

Цель исследования - изучить поведение пользователей "Яндекс.Музыки" на реальных данных для двух городов - Москвы и Санкт-Петербурга.

В ходе проведения исследования проверить следующие гипотезы:

1. Пользователи Москвы и Санкт-Петербурга по-разному слушают треки по понедельникам, средам и пятницам;
2. Утром в понедельник и вечером в пятницу у пользователей из Москвы преобладают одни жанры, у пользователей из Петербурга - другие. 
3. В Санкт-Петербурге наибольшей популярностью пользуется рэп, в Москве - поп-музыка.

### Изучение данных сервиса

До начала предобработки данных необходимо открыть и изучить данные. Для этого импортируем библиотеку pandas.

In [5]:
import pandas as pd #импорт библиотеки pandas

In [ ]:
df = pd.read_csv('/datasets/yandex_music_project.csv')# чтение файла с данными и сохранение в df

In [ ]:
df = pd.read_csv('/datasets/yandex_music_project.csv')# чтение файла с данными и сохранение в df

In [ ]:
df.head(10)# получение первых 10 строк таблицы df

С помощью метода info() получим данные о датасете.

In [4]:
df.info()# получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
  userID    65079 non-null object
Track       63848 non-null object
artist      57876 non-null object
genre       63881 non-null object
  City      65079 non-null object
time        65079 non-null object
Day         65079 non-null object
dtypes: object(7)
memory usage: 3.5+ MB


Исходя из информации о таблице мы видим, что все данные в датасете имеют тип 'object'.
В названиях колонок имеются нарушения хорошего стиля:
- встречаются пробелы;
- нарушения правил "змеиного регистра" при присваивании имени столбцу, состоящего из нескольких слов ('userID' вместо 'user_id'). Также количество значений в столбцах не одинаково, что говорит о наличии пропущенных значений. 


**Выводы**

Изучив данные выснилось, что датасет содержит семь колонок, которые содержат данные о:
- id пользователя;
- названии трека;
- наименовании исполнителя;
- жанре;
- городе;
- времени прослушивания;
- дне недели.

Однако для того, чтобы проверить гипотезы, необходимо предобработать данные, поскольку в них есть пропущенные значния, а также в наименованиях столбцов замечены нарушения хорошего стиля.

## Предобработка данных

Нам необходимо обработать пропуски, проверить данные на наличие дубликатов и, при необходимости, обработать их.

### Стиль заголовков
Выведем названия столбцов на экран:

In [5]:
display(df.columns)# перечень названий столбцов таблицы df

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Приведем названия колонок в порядок в соотвествии с правилами хорошего стиля: удалим пробелы, заглавные буквы и приведем названия в порядок по правилу "змеиного регистра".

In [6]:
df = df.rename(columns={'  userID':'user_id',
                        'Track':'track',
                        '  City  ':'city',
                        'Day':'day',
                        '  userID':'user_id'
                       })
                        # переименование столбцов

Проверим результат переименования:

In [7]:
display(df.columns)# проверка результатов - перечень названий столбцов

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Пропуски значений
Проверим данные на наличие пропущенных значений:

In [8]:
df.isna().sum()# подсчёт пропусков

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Не все пропущенные значения влияют на исследование. Так в `track` и `artist` пропуски не важны для работы. Достаточно заменить их явными обозначениями.

Но пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге, поэтому заполним пропуски явнями обозначениями.

Заменим пропущенные значения в столбцах `track`, `artist` и `genre` на строку `'unknown'`. Для этого создадим список `columns_to_replace`, переберем его элементы циклом `for` и для каждого столбца выполним замену пропущенных значений:

In [9]:
columns_to_replace = ['track', 'artist', 'genre']
for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown')

Убедимся, что в таблице не осталось пропусков. Для этого ещё раз посчитаем пропущенные значения.

In [10]:
df.isna().sum()# подсчёт пропусков

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Дубликаты
Теперь посчитам количество явных дубликатов:

In [11]:
df.duplicated().sum()# подсчёт явных дубликатов

3826

Используем метод drop_duplicates() `pandas` для удаления дубликатов:

In [12]:
df = df.drop_duplicates().reset_index(drop=True) #удаление явных дубликатов (с удалением старых индексов и формированием новых)

Убедимся, что полностью избавились от дубликатов:

In [13]:
df.duplicated().sum()# проверка на отсутствие дубликатов

0

Также избавимся от неявных дубликатов в колонке `genre`. Например, название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования.

Для этого выведем на экран список уникальных названий жанров, отсортированный в алфавитном порядке: извлечем нужный столбец датафрейма, применим метод сортировки к нему, для отсортированного столбца вызовем метод, который вернет уникальные значения столбца. 

Посмотрим, что получилось:

In [14]:
display(df['genre'].sort_values().unique())# Просмотр уникальных названий жанров

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Из списка названий жанров мы видим, что в жанре `hiphop` есть неявные дубликаты в написании: 
* *hip*,
* *hop*,
* *hip-hop*.

Необходимо удалить эти дубликаты, заменив верным написанием жанра. Для этого применим метод `replace_wrong_genres()` с двумя параметрами: 
* `wrong_genres` — список дубликатов,
* `correct_genre` — строка с правильным значением.


In [15]:
def replace_wrong_values(wrong_genres,correct_genre):
    for wrong_genre in wrong_genres:
        df['genre'] = df['genre'].replace(wrong_genre,correct_genre)# Функция для замены неявных дубликатов

Вызовем `replace_wrong_genres()` и передадим ей такие аргументы, чтобы она устранила неявные дубликаты: вместо `hip`, `hop` и `hip-hop` в таблице должно быть значение `hiphop`:

In [16]:
wrong_genres = ['hip','hop','hip-hop']
correct_genre = 'hiphop'
replace_wrong_values(wrong_genres,correct_genre)
# Устранение неявных дубликатов

Проверим, что заменили неправильные названия:

*   hip
*   hop
*   hip-hop

Выведем отсортированный список уникальных значений столбца `genre`:

In [17]:
display(df['genre'].sort_values().unique())# Проверка на неявные дубликаты

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Мы подкорректировали заголовки, чтобы упростить работу с таблицей. Без дубликатов исследование станет более точным.

Пропущенные значения были заменены на `'unknown'`. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке `genre`.

Теперь можно перейти к проверке гипотез. 

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Нам нужно проверить это по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.


Оценим активность пользователей в каждом городе, сгруппировав данные по городу и подсчитав значения.



In [18]:
display(df.groupby('city').count())

# Подсчёт прослушиваний в каждом городе

,user_id,track,artist,genre,time,day
city,,,,,,
Moscow,42741,42741,42741,42741,42741,42741
Saint-Petersburg,18512,18512,18512,18512,18512,18512


В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Теперь сгруппируем данные по дню недели и подсчитаем прослушивания в понедельник, среду и пятницу (в данных есть информация только по этим дням).


In [19]:
display(df.groupby('day').count())

# Подсчёт прослушиваний в каждый из трёх дней

,user_id,track,artist,genre,city,time
day,,,,,,
Friday,21840,21840,21840,21840,21840,21840
Monday,21354,21354,21354,21354,21354,21354
Wednesday,18059,18059,18059,18059,18059,18059


В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

В целом, пользователи и Москвы, и Петербурга менее активны по средам. Но необходимо рассмотреть данные по каждому городу в отдельности.

Теперь нам нужно создать функцию `number_tracks()`, которая посчитает прослушивания для заданного дня и города. Функция принимает на вход два параметра:
* день недели,
* название города.

В функции сохраним в переменную строки исходной таблицы, у которых значение:
  * в колонке `day` равно параметру `day`,
  * в колонке `city` равно параметру `city`.

Сделаем это с помощью фильтрации с логической индексацией.

Затем посчитаем значения в столбце `user_id` получившейся таблицы. Результат сохраним в новую переменную. Вернем результат работы функции.

In [20]:
def number_tracks(day,city):
    track_list = df[df['day']==day]
    track_list = track_list[track_list['city']==city]
    track_list_count = track_list['user_id'].count()
    return track_list_count
# <создание функции number_tracks()>
# Объявляется функция с двумя параметрами: day, city.
# В переменной track_list сохраняются те строки таблицы df, для которых 
# значение в столбце 'day' равно параметру day и одновременно значение
# в столбце 'city' равно параметру city (используйте последовательную фильтрацию
# с помощью логической индексации).
# В переменной track_list_count сохраняется число значений столбца 'user_id',
# рассчитанное методом count() для таблицы track_list.
# Функция возвращает число - значение track_list_count.
# Функция для подсчёта прослушиваний для конкретного города и дня.
# С помощью последовательной фильтрации с логической индексацией она 
# сначала получит из исходной таблицы строки с нужным днём,
# затем из результата отфильтрует строки с нужным городом,
# методом count() посчитает количество значений в колонке user_id. 
# Это количество функция вернёт в качестве результата

Вызовем `number_tracks()` шесть раз, меняя значение параметров — так, чтобы получить данные для каждого города в каждый из трёх дней.

In [21]:
number_tracks(day='Monday', city='Moscow')# количество прослушиваний в Москве по понедельникам

15740

In [22]:
number_tracks(day='Monday', city='Saint-Petersburg')# количество прослушиваний в Санкт-Петербурге по понедельникам

5614

In [23]:
number_tracks(day='Wednesday', city='Moscow')# количество прослушиваний в Москве по средам

11056

In [24]:
number_tracks(day='Wednesday', city='Saint-Petersburg')# количество прослушиваний в Санкт-Петербурге по средам

7003

In [25]:
number_tracks(day='Friday', city='Moscow')# количество прослушиваний в Москве по пятницам

15945

In [26]:
number_tracks(day='Friday', city='Saint-Petersburg')# количество прослушиваний в Санкт-Петербурге по пятницам

5895

При помощи конструктора pd.DataFrame создадим датафрейм, где:
* названия колонок — `['city', 'monday', 'wednesday', 'friday']`;
* данные — результаты, которые вы получили с помощью `number_tracks`.

In [27]:
columns_name = ['city','monday','wednesday','friday']
number_tracks = [['Moscow', 15740, 11056, 15945],
                 ['Saint-Petersburg', 5614, 7003, 5895],
                ]
df_final = pd.DataFrame(data = number_tracks, columns = columns_name)
display(df_final)# Таблица с результатами

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, в среду пользователи менее активны;
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь уступает автивности по средам.

А это значит, что первая гипотеза верна.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраним таблицы с данными в две переменные:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [28]:
moscow_general = df[df['city']=='Moscow']
display(moscow_general)# получение таблицы moscow_general из тех строк таблицы df, 
# для которых значение в столбце 'city' равно 'Moscow'


,user_id,track,artist,genre,city,time,day
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,unknown,ruspop,Moscow,09:17:40,Friday
...,...,...,...,...,...,...,...
61247,83A474E7,I Worship Only What You Bleed,The Black Dahlia Murder,extrememetal,Moscow,21:07:12,Monday
61248,729CBB09,My Name,McLean,rnb,Moscow,13:32:28,Wednesday
61250,C5E3A0D5,Jalopiina,unknown,industrial,Moscow,20:09:26,Friday
61251,321D0506,Freight Train,Chas McDevitt,rock,Moscow,21:43:59,Friday


In [29]:
spb_general = df[df['city']=='Saint-Petersburg']
display(spb_general)# получение таблицы spb_general из тех строк таблицы df,
# для которых значение в столбце 'city' равно 'Saint-Petersburg'


,user_id,track,artist,genre,city,time,day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
9,E772D5C0,Pessimist,unknown,dance,Saint-Petersburg,21:20:49,Wednesday
...,...,...,...,...,...,...,...
61239,D94F810B,Theme from the Walking Dead,Proyecto Halloween,film,Saint-Petersburg,21:14:40,Monday
61240,BC8EC5CF,Red Lips: Gta (Rover Rework),Rover,electronic,Saint-Petersburg,21:06:50,Monday
61241,29E04611,Bre Petrunko,Perunika Trio,world,Saint-Petersburg,13:56:00,Monday
61242,1B91C621,(Hello) Cloud Mountain,sleepmakeswaves,postrock,Saint-Petersburg,09:22:13,Monday


Нам нужно создать функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция должна вернуть информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [30]:
def genre_weekday(table,day,time1,time2):
    genre_df = table[table['day'] == day]
    genre_df = genre_df[genre_df['time'] > time1]
    genre_df = genre_df[genre_df['time'] < time2]
    genre_df_sorted = genre_df['genre'].value_counts()
    return genre_df_sorted.head(10)



# Объявление функции genre_weekday() с параметрами table, day, time1, time2,
# которая возвращает информацию о самых популярных жанрах в указанный день в
# заданное время:
# 1) в переменную genre_df сохраняются те строки переданного датафрейма table, для
#    которых одновременно:
#    - значение в столбце day равно значению аргумента day
#    - значение в столбце time больше значения аргумента time1
#    - значение в столбце time меньше значения аргумента time2
#    Используйте последовательную фильтрацию с помощью логической индексации.
# 2) сгруппировать датафрейм genre_df по столбцу genre, взять один из его
#    столбцов и посчитать методом count() количество записей для каждого из
#    присутствующих жанров, получившийся Series записать в переменную
#    genre_df_count
# 3) отсортировать genre_df_count по убыванию встречаемости и сохранить
#    в переменную genre_df_sorted
# 4) вернуть Series из 10 первых значений genre_df_sorted, это будут топ-10
#    популярных жанров (в указанный день, в заданное время)

Теперь сравним резульаты работы функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [31]:
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')
# вызов функции для утра понедельника в Москве (вместо df — таблица moscow_general)
# объекты, хранящие время, являются строками и сравниваются как строки
# пример вызова: genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [32]:
genre_weekday(spb_general, 'Monday', '07:00', '11:00')
# вызов функции для утра понедельника в Петербурге (вместо df — таблица spb_general)

pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [33]:
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')# вызов функции для вечера пятницы в Москве

pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
classical      163
alternative    163
rusrap         142
Name: genre, dtype: int64

In [34]:
genre_weekday(spb_general, 'Friday', '17:00', '23:00')# вызов функции для вечера пятницы в Петербурге

pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы никак не влияет на топ-10 жанров - он остается таким же, только некоторые показатели становятся выше, чем другие.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Возможно, пропуски в данных влияют на этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Для проверки гипотезы нужно сгруппировать данные датасета `moscow_general` по жанру и посчитать прослушивания треков каждого жанра методом `count()`, отсортировать результат в порядке убывания и сохранить его в таблице `moscow_genres`.

In [35]:
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending = False)
# одной строкой: группировка таблицы moscow_general по столбцу 'genre', 
# подсчёт числа значений 'genre' в этой группировке методом count(), 
# сортировка получившегося Series в порядке убывания и сохранение в moscow_genres

Просмотрим первые десять строк `moscow_genres`:

In [36]:
moscow_genres.head(10)# просмотр первых 10 строк moscow_genres

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Проделаем то же самое для данных по Петербургу:

In [37]:
spb_genres = spb_general.groupby('genre')['time'].count().sort_values(ascending = False)
# одной строкой: группировка таблицы spb_general по столбцу 'genre', 
# подсчёт числа значений 'genre' в этой группировке методом count(), 
# сортировка получившегося Series в порядке убывания и сохранение в spb_genres

In [44]:
type(spb_general.groupby('genre')['time'].count())

pandas.core.series.Series

Просмотрим первые десять строк `spb_genres`:

In [38]:
spb_genres.head(10)# просмотр первых 10 строк spb_genres

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: time, dtype: int64

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* А вот рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

Проверив три гипотезы, мы можем сделать следующие выводы:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — как в Москве, так и в Петербурге. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. В целом, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Возможно различия в предпочтениях и существуют, на основной массе пользователей они незаметны.